# Train a Logistic Classifier on MNIST Data

In this example we will load labels and pointers to the data into a Gota dataframe.

In [1]:
import (
    "fmt"
     mnist "github.com/petar/GoMNIST"
    "github.com/kniren/gota/dataframe"
    "github.com/kniren/gota/series"
    "math/rand"
    "github.com/cdipaolo/goml/linear"
    "github.com/cdipaolo/goml/base"
    "image"
    "bytes"
    "math"
    "github.com/gonum/stat"
    "github.com/gonum/integrate"
)

In [2]:
set, err := mnist.ReadSet("../datasets/mnist/images.gz", "../datasets/mnist/labels.gz")

In [3]:
set.Images[1]

In [4]:
func MNISTSetToDataframe(st *mnist.Set, maxExamples int) dataframe.DataFrame {
    length := maxExamples
    if length > len(st.Images) {
        length = len(st.Images)
    }
    s := make([]string, length, length)
    l := make([]int, length, length)
    for i := 0; i < length; i++ {
        s[i] = string(st.Images[i])
        l[i] = int(st.Labels[i])
    }
    var df dataframe.DataFrame
    images := series.Strings(s)
    images.Name = "Image"
    labels := series.Ints(l)
    labels.Name = "Label"
    df = dataframe.New(images, labels)
    return df
}

In [5]:
df := MNISTSetToDataframe(set, 1000)

In [6]:
categories := []string{"tshirt", "trouser", "pullover", "dress", "coat", "sandal", "shirt", "shoe", "bag", "boot"}

In [7]:
func Split(df dataframe.DataFrame, valFraction float64) (training dataframe.DataFrame, validation dataframe.DataFrame){
    perm := rand.Perm(df.Nrow())
    cutoff := int(valFraction*float64(len(perm)))
    training = df.Subset(perm[:cutoff])
    validation = df.Subset(perm[cutoff:])
    return training, validation
}

In [8]:
training, validation := Split(df, 0.75)

In [9]:
func EqualsInt(s series.Series, to int) (*series.Series, error) {
    eq := make([]int, s.Len(), s.Len())
    ints, err := s.Int()
    if err != nil {
        return nil, err
    }
    for i := range ints {
        if ints[i] == to {
            eq[i] = 1
        }
    }
    ret := series.Ints(eq)
    return &ret, nil
}

In [10]:
trainingIsTrouser, err1 := EqualsInt(training.Col("Label"), 1)
validationIsTrouser, err2 := EqualsInt(validation.Col("Label"), 1)
if err1 != nil || err2 != nil {
    fmt.Println("Error", err1, err2)
}


In [11]:
func NormalizeBytes(bs []byte) []float64 {
    ret := make([]float64, len(bs), len(bs))
    for i := range bs {
        ret[i] = float64(bs[i])/255.
    }
    return ret
}

In [12]:
func ImageSeriesToFloats(df dataframe.DataFrame, col string) [][]float64 {
    s := df.Col(col)
    ret := make([][]float64, s.Len(), s.Len())
    for i := 0; i < s.Len(); i++ {
        b := []byte(s.Elem(i).String())
        ret[i] = NormalizeBytes(b)
    }
    return ret
}

In [13]:
trainingImages := ImageSeriesToFloats(training, "Image")
validationImages := ImageSeriesToFloats(validation, "Image")

## Logistic Classifier for a single class

In [14]:
model := linear.NewLogistic(base.BatchGA, 1e-4, 1, 150, trainingImages, trainingIsTrouser.Float())

In [15]:
//Train
err := model.Learn()
if err != nil {
    fmt.Println(err)
}

In [16]:
//Count correct classifications
var correct = 0.
for i := range validationImages {
    prediction, err := model.Predict(validationImages[i])
    if err != nil {
        panic(err)
    }
    
    if math.Round(prediction[0]) == validationIsTrouser.Elem(i).Float() {
        correct++
    }
}

In [17]:
//accuracy
correct/float64(len(validationImages))

0.988

In [18]:
//Count true positives and false negatives
var truePositives = 0.
var falsePositives = 0.
var falseNegatives = 0.
for i := range validationImages {
    prediction, err := model.Predict(validationImages[i])
    if err != nil {
        panic(err)
    }
    if validationIsTrouser.Elem(i).Float() == 1 {
        if math.Round(prediction[0]) == 0 {
            // Predicted false, but actually true
            falseNegatives ++
        } else {
            // Predicted true, correctly
            truePositives ++
        }
    } else {
        if math.Round(prediction[0]) == 1 {
            // Predicted true, but actually false
            falsePositives ++
        }
    }
}

In [19]:
//precision
truePositives/(truePositives+falsePositives)

1

In [20]:
//recall
truePositives/(truePositives+falseNegatives)

0.9032258064516129

## Using the Softmax model for multi-class classification

In [21]:
model2 := linear.NewSoftmax(base.BatchGA, 1e-4, 1, 10, 100, trainingImages, training.Col("Label").Float())

In [22]:
//Train
err := model2.Learn()
if err != nil {
    fmt.Println(err)
}

In [23]:
func MaxIndex(f []float64) (i int){
    var (
        curr float64
        ix int = -1
    )
    for i := range f {
        if f[i] > curr {
            curr = f[i]
            ix = i
        }
    }
    return ix
}

In [24]:
//create objects for ROC generation
//as per https://godoc.org/github.com/gonum/stat#ROC
y := make([][]float64, len(categories), len(categories))
classes := make([][]bool, len(categories), len(categories))
//Validate
for i := 0; i < validation.Col("Image").Len(); i++ {
    prediction, err := model2.Predict(validationImages[i])
    if err != nil {
        panic(err)
    }
    for j := range categories {
        y[j] = append(y[j], prediction[j])
        classes[j] = append(classes[j], validation.Col("Label").Elem(i).Float() != float64(j))
    }
}

In [25]:
//Calculate ROC
tprs := make([][]float64, len(categories), len(categories))
fprs := make([][]float64, len(categories), len(categories))

for i := range categories {
    stat.SortWeightedLabeled(y[i], classes[i], nil)
    tprs[i], fprs[i] = stat.ROC(0, y[i], classes[i], nil)
}

In [26]:
for i := range categories {
    fmt.Println(categories[i])
    auc := integrate.Trapezoidal(fprs[i], tprs[i])
    fmt.Println(auc)
}

tshirt
0.9189451938267107
trouser
0.9577257328030637
pullover
0.6114433811802232
dress
0.8977530926533703
coat
0.9302815552576138
sandal
0.961950330371383
shirt
0.5922229153670202
shoe
0.9562527618205923
bag
0.9096607669616519
boot
0.9862095384026048


In [27]:
import (
    "gonum.org/v1/plot"
    "gonum.org/v1/plot/plotter"
    "gonum.org/v1/plot/plotutil"
    "gonum.org/v1/plot/vg"
    "bufio"
)

In [28]:
func plotROCBytes(fprs, tprs [][]float64, labels []string) []byte {
    p, err := plot.New()
    if err != nil {
        panic(err)
    }

    p.Title.Text = "ROC Curves"
    p.X.Label.Text = "False Positive Rate"
    p.Y.Label.Text = "True Positive Rate"
    
    for i := range labels {
        pts := make(plotter.XYs, len(fprs[i]))
        for j := range fprs[i] {
            pts[j].X = fprs[i][j]
            pts[j].Y = tprs[i][j]
        }
        lines, points, err := plotter.NewLinePoints(pts)
        if err != nil {
            panic(err)
        }
        lines.Color = plotutil.Color(i)
        lines.Width = 2
        points.Shape = nil

        p.Add(lines, points)
        p.Legend.Add(labels[i], lines, points)
    }

    w, err := p.WriterTo(5*vg.Inch, 4*vg.Inch, "jpg")
    if err != nil{
        panic(err)
    }
    if err := p.Save(5*vg.Inch, 4*vg.Inch, "Multi-class ROC.jpg"); err != nil {
        panic(err)
    }
    var b bytes.Buffer
    writer := bufio.NewWriter(&b)
    w.WriteTo(writer)
    return b.Bytes()
}

In [29]:
display.JPEG(plotROCBytes(fprs, tprs, categories))